# Waste generation and scenario planning exploration

We explore the partitioned data sets and see how easy it is to merge the files with previous household to road-network matchings using lat-lon coordinates.

To easily snap the lat-lon coordinates, we just extract all unique lat-lon coordinates from all files and match them once.

In [55]:
%%html
<style>
table {float:left}
</style>

| Document info | |
| --- | --- | 
| Area of interest: | Cape Town |
| Planning type: | All REL type producers |
| Prepared by: | Waste Labs (wastelabs.co) |
| Prepared for: | Johan W. Joubert |
| Contact: | elias@wastelabs.co |

In [3]:
%reload_kedro
import boto3
import geopandas as gpd
import numpy as np
import pandas as pd
import yaml
from GPSOdyssey import Kepler

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

2022-05-17 11:33:06,320 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-05-17 11:33:06,379 - root - INFO - ** Kedro project Demand estimation and waste collection routing optimisation for the City of Cape Town
2022-05-17 11:33:06,380 - root - INFO - Defined global variable `context`, `session` and `catalog`
2022-05-17 11:33:06,390 - root - INFO - Registered line magic `run_viz`


## Load sample files

In [52]:
test_file = "households_001"
local_pop1 = catalog.load("syn_pop_scenarios_local")[test_file]()
local_waste1 = catalog.load("waste_gen_scenarios_local")[test_file]()

test_file = "households_002"
local_pop2 = catalog.load("syn_pop_scenarios_local")[test_file]()
local_waste2 = catalog.load("waste_gen_scenarios_local")[test_file]()

2022-05-17 11:53:23,349 - kedro.io.data_catalog - INFO - Loading data from `syn_pop_scenarios_local` (PartitionedDataSet)...
2022-05-17 11:53:23,428 - kedro.io.data_catalog - INFO - Loading data from `waste_gen_scenarios_local` (PartitionedDataSet)...
2022-05-17 11:53:23,444 - kedro.io.data_catalog - INFO - Loading data from `syn_pop_scenarios_local` (PartitionedDataSet)...
2022-05-17 11:53:23,513 - kedro.io.data_catalog - INFO - Loading data from `waste_gen_scenarios_local` (PartitionedDataSet)...


In [53]:
local_pop1.shape
local_waste1.shape

(38603, 14)

(38603, 2)

In [54]:
local_pop2.shape
local_waste2.shape

(38655, 14)

(38655, 2)

## Compare lat-lon coordinates

In [50]:
local_pop1["lon_lat_merge_id"] = (
    local_pop1[["parcelLon", "parcelLat"]].round(6).astype(str).agg("_".join, axis=1)
)
local_pop2["lon_lat_merge_id"] = (
    local_pop1[["parcelLon", "parcelLat"]].round(6).astype(str).agg("_".join, axis=1)
)

In [51]:
local_pop1["lon_lat_merge_id"].isin(local_pop2["lon_lat_merge_id"]).sum()

38697